In [1]:
import pandas as pd
import numpy as np
import duckdb
import os
import sys

current_dir = os.getcwd()
parent_dir = os.path.abspath(os.path.join(current_dir, "../.."))
sys.path.append(parent_dir)

In [2]:
import constants
all_countries = set(constants.eu + constants.americas + constants.asia + constants.africa + constants.oceania)

In [3]:
eu27_countries = [
    "AT", "BE", "BG", "HR", "CY", "CZ", 
    "DK", "EE", "FI", "FR", "DE", "GR", 
    "HU", "IE", "IT", "LV", "LT", "LU", 
    "MT", "NL", "PL", "PT", "RO", "SK", 
    "SI", "ES", "SE"
]

balkans = [
    "BA", "XK", "MK", "CS", "AL"
]

KEY_FINANCIALS_DETAILED_INDUSTRY_PATH = "..\\..\\data_raw\\key_financials_detailed\\industry_global_financials_and_ratios_eur"
KEY_FINANCIALS_DETAILED_BANKSS_PATH = "..\\data_raw\\key_financials_detailed\\banks_global_financials_and_ratios_eur"
KEY_FINANCIALS_DETAILED_INSURANCES_PATH = "..\\data_raw\\key_financials_detailed\\insurances_global_financials_and_ratios_eur"

In [4]:
key_financials_detail_path = "..\\..\\data_raw\\key_financials_detailed\\industry_global_financials_and_ratios_eur\\"

def fetch_data_for_country(country_iso_code):
    query = f"""
    SELECT 
        bvd_id_number,
        operating_revenue_turnover_,
        number_of_employees,
        costs_of_goods_sold,
        material_costs,
        added_value,
        EXTRACT(YEAR FROM closing_date) AS year  
    FROM 
        parquet_scan('{key_financials_detail_path}*.parquet')
    WHERE
        CAST(bvd_id_number AS STRING) LIKE '{country_iso_code}%'
    ORDER BY 
        bvd_id_number, year
    """

    conn = duckdb.connect()
    df = conn.execute(query).fetchdf()

    return df


In [5]:
# TEST
it = fetch_data_for_country("IT")
# ba = fetch_data_for_country("BA")
# cs = fetch_data_for_country("CS")
# mk = fetch_data_for_country("MK")
# xk = fetch_data_for_country("XK")
# it = fetch_data_for_country("IT")


In [ ]:
import os

output_path = "..\\..\\data_processed\\key_financials_detailed_processed\\"

# Ensure the directory exists
os.makedirs(output_path, exist_ok=True)

for country in all_countries:
    df = fetch_data_for_country(country)
    df.to_parquet(f"{output_path}key_financials_detailed_{country}.parquet")
    print(f"Saved: {output_path}key_financials_detailed_{country}.parquet")


In [8]:
it.columns

Index(['bvd_id_number', 'operating_revenue_turnover_', 'number_of_employees',
       'costs_of_goods_sold', 'material_costs', 'added_value', 'year'],
      dtype='object')

In [6]:
## TEST 

firmographics_ita = pd.read_parquet("..\\..\\data_processed\\firmographics_processed\\firmographics_IT.parquet")

In [10]:
merged_df = pd.merge(
    firmographics_ita,
    it,
    left_on="bvd_id_number",
    right_on="bvd_id_number",
    how="outer",          # Use outer join to see all rows
    indicator=True        # Adds the '_merge' column
)


In [11]:
merged_df.value_counts("_merge")

_merge
both          75223744
left_only      1280343
right_only      151814
Name: count, dtype: int64